# Answering questions using Roberta

## Main solution using roberta

In [60]:
!pwd

/home/george/Documents/LeWagon/Transformers_Hugging_Face


In [2]:
"""Install requirements"""
# Install the transformers library from HuggingFace
!pip install transformers torch pytesseract
# You'll also need some extra tools that some of these models use under the hood
! pip install sentencepiece sacremoses
# to read pdf
!pip install pdfquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 4.4 MB/s eta 0:00:00
  Using cached pytesseract-0.3.10-py3-none-any.whl (14 kB)
  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 8.5 MB/s eta 0:00:00
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-n

In [4]:
"""Import packages"""
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import pandas as pd

"""Import our question answering model"""
question_answerer = pipeline(model = 'deepset/roberta-base-squad2')

In [15]:
def answer_questions_with_confidence(context = "You did not specify any content", questions = ["Did you mean to specify a question?"]):
    """Takes a list called 'questions' that contains the questions to answer
    Takes some text called 'content' as a source for answering questions
    Returns a dataframe of the questions with their answers and an assessment of confidence in the answers
    If no context or content is provided, returns a dataframe requesting these"""
    
    # List to fill with questions, answers, and confidence
    questions_answers = []
    
    # For each question create an empty dictionary and call the question_answerer model on the question
    for q in questions: 
        q_a_dict = {}
        q_a = question_answerer(question=q, context=context)
        
        # Assign the question, and outputs of the question_answerer model to the dictionary
        q_a_dict['confidence'] = q_a['score']
        q_a_dict['question'] = q
        q_a_dict['answer'] = q_a['answer'].replace('\n', ' ')
        
        # Add the dictionary to the list and then convert the final list of dicts to a dataframe
        questions_answers.append(q_a_dict) 
    questions_answers = pd.DataFrame(questions_answers)
    
    # Set a large maxcolwidth to allow for potentially long answers
    pd.options.display.max_colwidth = 20000
    
    return questions_answers

In [10]:
def select_top_n_questions(context, questions, c = 0.5, n = 5):
    """Selects the top n questions with the highest confidence level c
    User can define how many questions are required and the minimum confidence level"""
    
    # Call answer_questions to get a df of questions and answers
    questions_answers = answer_questions_with_confidence(context, questions)
    
    # Filter for confidence
    conf_questions = questions_answers[questions_answers['confidence'] > c] 
    
    # Return n questions ordered by confidence
    selected_questions = conf_questions.sort_values(by='confidence', ascending=False).head(n)\
    .reset_index().rename(columns={'index':'original_question_number'}) 
    
    """Check whether enough questions can be returned and explain why if not"""
    
    # Were enough questions generated?
    if len(questions_answers) < n:
        print(f"Only {len(questions_answers)} questions were generated")
        
        # Did enough questions meet the confidence requirement?
        if len(selected_questions) == 0:
            print("No questions met your required confidence level.")
        elif len(selected_questions) < n: 
            print(f"Not enough questions met your required confidence level,\
 but here {'is' if len(selected_questions) == 1 else 'are'} the {len(selected_questions)} that did:")
        else:
            print(f"Here are your {n} questions")
            
    else:
        # Did enough questions meet the confidence requirement?
        if len(selected_questions) == 0:
            print("No questions met your required confidence level.")
        elif len(selected_questions) < n: 
            print(f"Not enough questions met your required confidence level,\
 but here {'is' if len(selected_questions) == 1 else 'are'} the {len(selected_questions)} that did:")
        else:
            print(f"Here are your {n} questions")
            
    return selected_questions

## Training model

In [ ]:
to_train = pipeline(model = 'deepset/roberta-base-squad2')

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

tokens = tokenizer.encode("This is easy!", return_tensors = "tf")

print(tokens)

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("deepset/roberta-base-squad2", from_pt = True)

output_tokens = model.generate(tokens)

print(output_tokens)

print(tokenizer.decode(output[0]))

## Sources

### PDF

In [ ]:
"""Open a pdf file as a possible source of context"""
from pdfquery import PDFQuery

pdf = PDFQuery('/home/george/Downloads/intelligent_M_and_A.pdf')
pdf.load()

# Use CSS-like selectors to locate the elements
text_elements = pdf.pq('LTTextLineHorizontal')

# Extract the text from the elements
text = [t.text for t in text_elements]

print(text)

### Text files

In [ ]:
"""Open a text file as a possible source of context"""
file = open("/home/george/Documents/example_article.txt", "r") # Example file
content = file.read()
print(content)
file.close()

question = ['Where is Spain?'] # Example question

### OCR

In [37]:
"""Extract text using OCR
This works with images obtained from the snipping tool.
This works with photographs of text but images need to be rotated to ensure they are the correct way up.
It does not recognise handwriting."""

# """For """
# !sudo apt install tesseract-ocr  
# !sudo apt install libtesseract-dev
# !pip install Pillow pytesseract

import pytesseract
from PIL import Image

def ocr_document(image_path):
    # Open the image using the Pillow library
    image = Image.open(image_path)

    # Perform OCR using Tesseract
    text = pytesseract.image_to_string(image)

    return text

In [40]:
# Example usage
image_path = '/home/george/Downloads/articletoocrquizachu/big.jpg'
result_text = ocr_document(image_path)

print("OCR Result:")
print(result_text)

OCR Result:
The startup’s flagship product, Pillow Talk, allows you to
connect to a loved one when you are unable to be in the same
place as them, all through the sound and feel of their heartbeat.

I ittle Riot make technology that facilitates human connection.

So how does it work? As a pair, each of you get a wristband to wear
to bed and a small speaker that should be placed under your pillow.
The wristband picks up your real-time heartbeat and sends it to the
other person's pillow. In your own pillow, you can hear the heartbeat
of your loved one, wherever in the world you each may be.

What started as a project at university, turned into the well-loved
Pillow Talk product which is now used in over 70 countries across
the globe.

Having won the ‘Biggest 2020 Hero’ award at Startup Magazine's first
ever Hustle Awards, we thought it was about time to check in with
Founder Joanna Montgomery to see what's new, where the startup is
now, and find out about the last few years.

HOW HAS THE

### Scraping

In [ ]:
"""Import packages"""
import requests
from bs4 import BeautifulSoup
import re

In [30]:
"""Scrape Wikipedia as a possible source of context"""

import requests
from bs4 import BeautifulSoup

def scrape_wikipedia_article(url):
    # Send an HTTP request to the URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the main article text (adjust the selector based on the structure of the webpage)
        article_text = soup.find('div', {'id': 'mw-content-text'}).get_text(separator='\n')
        
        return article_text
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

# Example usage
url = 'https://en.wikipedia.org/wiki/Ai_(chimpanzee)'
wikipedia_article_text = scrape_wikipedia_article(url)

if wikipedia_article_text:
    print(wikipedia_article_text)



Subject of the Ai project


Ai
Species
chimpanzee
Born
c.
 1976
 (age 
46–47)
Guinean Forests of West Africa
Offspring
Ayumu (chimpanzee)


Ai
 (born in 1976, estimated) is a female 
western chimpanzee
 (
Pan troglodytes verus
),
[1]
 currently living at the 
Primate Research Institute
 of 
Kyoto University
 (acronym KUPRI). She is the first subject of the 
Ai project
, a research program started in 1978 by Kiyoko Murofushi and 
Tetsuro Matsuzawa
 which is aimed at understanding chimpanzee 
cognition
 through computer interface experiments.
[2]






Biography
[
edit
]


Ai was born in 1976 (estimated), in the 
Guinean Forests of West Africa
.
[1]
 Born wild, Ai was soon taken into captivity and sold to KUPRI in 1977 by an animal trader (this type of sale became illegal in 1980 with 
Japan
's ratification of 
CITES
).
[1]
  She was the first subject of KUPRI’s chimpanzee project, which was intended to become Japan’s first ape-language study in the vein of earlier ape-language studies.


In [20]:
"""Scrape BBC as a possible source of context"""

import requests
from bs4 import BeautifulSoup

def scrape_bbc_article(url):
    # Send an HTTP request to the URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the main article text (adjust the selector based on the structure of the webpage)
        article = []
        for para in soup.find_all("div", {"data-component": "text-block"}):
            article.append(para.text)
        article_text = " ".join(article)
        
        return article_text
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

# Example usage - cat article
url = 'https://www.bbc.co.uk/news/uk-england-beds-bucks-herts-67407334'
bbc_article_text = scrape_bbc_article(url).replace('\n', ' ')

if bbc_article_text:
    print(bbc_article_text)


A cat whose pictures went viral for regularly visiting a railway station is releasing a Christmas single. Four-year-old Nala has been delighting commuters who have been taking photos of her at Stevenage station. Owner Natasha Ambler revealed the cat was releasing a single called Meow and has been approached for a book deal. The ginger tabby has also recorded a video for the song due to be released this week, under the name Nala the Station Cat. It has been produced by Danny Kirsch, who wrote it with Joe Killington, while Nala is also co-credited as a songwriter, as well as a vocalist. Ms Ambler said "we want to spread the happiness that Stevenage has had, and she's had on socials to the world". The single is officially released on Wednesday and BBC Three Counties Radio's Justin Dealey gave the single an exclusive first play on Sunday. "I'm slightly lost for words," said the presenter after the song finished. Nala's owner replied: "So am I to be fair." The musical cat does not yet have 

### Importing audio

In [ ]:
"""Installs to analyse audio"""
!sudo apt install ffmpeg
!pip3 install datasets
!pip install SoundFile
!pip install librosa

In [ ]:
"""Example audio to analyse"""
!mkdir data
!curl https://wagon-public-datasets.s3.amazonaws.com/deep_learning_datasets/harvard.wav > data/harvard.wav

In [ ]:
"""Packages for audio"""
from scipy.io import wavfile
from IPython.display import Audio

In [ ]:
"""Read the audio file and play it to verify"""
rate, audio = wavfile.read("data/harvard.wav")
Audio(audio.T, rate=rate)

In [ ]:
"""Transcription of a downloaded wav file"""

from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa  

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None

# Whisper requires a sampling rate of 16000 so must convert this with librosa
audio, rate = librosa.load('data/harvard.wav', sr=16000)
input_features = processor(audio, sampling_rate=rate, return_tensors="pt").input_features 

# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)


In [ ]:
"""Transcription of a flac file from hugging face"""

from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None

# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)


In [ ]:
transcription

### Reading handwriting

In [ ]:
"""This works well for single lines of handwriting but does not support multiple lines.
I need to split multiple line files into single lines."""

hw = pipeline(model = 'microsoft/trocr-base-handwritten')

In [59]:
"""This attempts to split images. It is the first time I gave up and got chatgpt to write code for me.
It does not work very well - it identifies words but does not link them correctly as lines."""

import cv2
import os
import pytesseract

def split_and_save_handwritten_lines(image_path, output_directory):
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use adaptive thresholding to preprocess the image
    _, binary_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # List to store individual line images
    line_images = []

    # Minimum width and height threshold for a contour to be considered a line
    min_width_threshold = 300
    min_height_threshold = 20

    # Iterate through contours
    for i, contour in enumerate(contours):
        # Get bounding box for each contour
        x, y, w, h = cv2.boundingRect(contour)

        # Filter out contours based on width and height
        if w > min_width_threshold and h > min_height_threshold:
            # Crop the original image to extract the line
            line_image = image[y:y+h, x:x+w]

            # Save the line image to the output directory
            output_path = os.path.join(output_directory, f'line_{i+1}.png')
            cv2.imwrite(output_path, line_image)

            # Append the line image to the list
            line_images.append(line_image)

    return line_images

# Example usage
image_path = '/home/george/Downloads/Handwriting-Y4.png'
output_directory = '/home/george/Downloads/split_text'
lines = split_and_save_handwritten_lines(image_path, output_directory)

# Print the paths of saved line images
for i, line_image in enumerate(lines, start=1):
    print(f"Saved line {i} to {os.path.join(output_directory, f'line_{i}.png')}")


Saved line 1 to /home/george/Downloads/split_text/line_1.png


In [13]:
"""Question-answer format"""
ocr(image='/home/george/Downloads/magnification.jpg',question="What does this page say?")

[{'score': 0.7278719544410706,
  'answer': 'Calculating the size of an object',
  'start': 0,
  'end': 5}]

### Text in more complex layouts, eg invoices or posters

In [ ]:
"""First model - this answers questions about documents
- this works for very simple documents 
but struggles for anything which implies relationships (e.g. two text boxes that relate to one another)"""
ocr = pipeline(model = 'impira/layoutlm-invoices') #This struggles to find relationships between objects

## Tests

In [ ]:
"""Tested distilbert to see if quicker. Some gains but less accurate"""
"""In tests, 2 means distilbert passed, 1 means roberta passed"""
question_answerer2 = pipeline(model = 'distilbert-base-uncased-distilled-squad') 

#### Underground questions

In [ ]:
underground_context = """The history of the London Underground began in the 19th century with the construction of the Metropolitan Railway,
the world's first underground railway. The Metropolitan Railway, which opened in 1863 using gas-lit wooden carriages hauled by steam 
locomotives, worked with the District Railway to complete London's Circle line in 1884. Both railways expanded, the Metropolitan eventually 
extending as far as Verney Junction in Buckinghamshire, more than 50 miles (80 km) from Baker Street and the centre of London. 
The first deep-level tube line, the City and South London Railway, opened in 1890 with electric trains. 
This was followed by the Waterloo & City Railway in 1898, the Central London Railway in 1900, and the Great Northern and City Railway 
in 1904. The Underground Electric Railways Company of London (UERL) was established in 1902 to fund the electrification of the 
District Railway and to complete and operate three tube lines, the Baker Street and Waterloo Railway, the Charing Cross, Euston and 
Hampstead Railway and the Great Northern, Piccadilly and Brompton Railway, which opened in 1906–07. By 1907 the District and 
Metropolitan Railways had electrified the underground sections of their lines.

Under a joint marketing agreement between most of the companies in the early years of the 20th century, UNDERGROUND signs appeared 
outside stations in central London. World War I delayed extensions of the Bakerloo and Central London Railways, and people used the 
tube stations as shelters during Zeppelin air raids by June 1915. After the war, government-backed financial guarantees were used to 
expand the network, and the tunnels of the City and South London and Charing Cross, Euston and Hampstead Railways were linked at Euston and 
Kennington, although the combined service was not named the Northern line until later. The Piccadilly line was extended north to 
Cockfosters and took over District line branches to Harrow (later Uxbridge) and Hounslow. In 1933, the underground railways and all London 
area tram and bus operators were merged into the London Passenger Transport Board (LPTB). The outlying branches of the Metropolitan were 
closed; various upgrades were planned. The Bakerloo line's extension to take over the Metropolitan's Stanmore branch, and extensions of 
the Central and Northern lines, formed part of the 1930s New Works Programme. The outbreak of World War II in 1939 halted or interrupted 
some of this work, and many tube stations were used as air-raid shelters. """

In [ ]:
underground_questions = ["In what year did the London Underground begin?", # 1963
"In what year did the UERL begin to be built?", # 1902 - 2, 1
"In what year did the London Underground begin?",
"What railway line was built in the 18th century?",
"In what year did the Thames & City Railway open?",
"How many kilometers long did it take to travel through London's Station?",
"When did the London Underground first operate?",
"In what year was the London Underground Station opened?",
"How many kilometers long was the London Underground Station?",
"In what year did the Thames & City Railway open?",
"What railway line was built in London in the 18th century?",
"What railway line was built in the London Underground?",
"When did the UERL begin to run through the London Underground?",
"When did the London Underground begin?",
"What was the name of the first railway station in London?",
"In what year did the London Underground begin?",
"In what year was the London Underground Station closed?",
"What railway line opened in 1880?",
"What railway line was built in London in 1880?",
"What railway line was built in the 18th century?"]

In [17]:
answer_questions_with_confidence2(underground_context, underground_questions) # underground qs using distilbert approx 15 seconds

,confidence,question,answer
0,0.930403,In what year did the London Underground begin?,1933
1,0.996830,In what year did the UERL begin to be built?,1902
2,0.930403,In what year did the London Underground begin?,1933
3,0.549939,What railway line was built in the 18th century?,Metropolitan Railway
4,0.881689,In what year did the Thames & City Railway open?,1898
5,0.771743,How many kilometers long did it take to travel through London's Station?,80
6,0.830885,When did the London Underground first operate?,1933
7,0.914037,In what year was the London Underground Station opened?,1933
8,0.794559,How many kilometers long was the London Underground Station?,80
9,0.881689,In what year did the Thames & City Railway open?,1898


In [18]:
answer_questions_with_confidence(underground_context, underground_questions) #using roberta approx 35 seconds

,confidence,question,answer
0,2.551449e-01,In what year did the London Underground begin?,19th century
1,9.581340e-01,In what year did the UERL begin to be built?,1902
2,2.551449e-01,In what year did the London Underground begin?,19th century
3,4.702023e-07,What railway line was built in the 18th century?,Piccadilly
4,2.806922e-06,In what year did the Thames & City Railway open?,1898
5,1.153477e-01,How many kilometers long did it take to travel through London's Station?,80
6,2.953299e-01,When did the London Underground first operate?,19th century
7,2.517956e-01,In what year was the London Underground Station opened?,1863
8,5.730436e-01,How many kilometers long was the London Underground Station?,80
9,2.806922e-06,In what year did the Thames & City Railway open?,1898


#### Cat questions

In [21]:
bbc_article_text

'A cat whose pictures went viral for regularly visiting a railway station is releasing a Christmas single. Four-year-old Nala has been delighting commuters who have been taking photos of her at Stevenage station. Owner Natasha Ambler revealed the cat was releasing a single called Meow and has been approached for a book deal. The ginger tabby has also recorded a video for the song due to be released this week, under the name Nala the Station Cat. It has been produced by Danny Kirsch, who wrote it with Joe Killington, while Nala is also co-credited as a songwriter, as well as a vocalist. Ms Ambler said "we want to spread the happiness that Stevenage has had, and she\'s had on socials to the world". The single is officially released on Wednesday and BBC Three Counties Radio\'s Justin Dealey gave the single an exclusive first play on Sunday. "I\'m slightly lost for words," said the presenter after the song finished. Nala\'s owner replied: "So am I to be fair." The musical cat does not yet 

In [ ]:
# Example questions - cat article
bbc_cat_questions = ['Where will profit go?','Who produced the song?','What is the song called?',\
             'Who gave the song its first play?','When will the song be released?','Who wrote the song?',\
             'Where was the video filmed?','How has nala been delighting commuters?',\
             "Who's pictures went viral?", 'All proceeds from the single will be what?', 'What links Danny and Joe?']

In [22]:
bbc_cat_questions # 2 if passed by 2, 1 if passed by 1
# RSPCA and Stevenage homelessness charity Feed Up Warm Up 2, 1
# Danny Kirsch 2, 1
# Meow 2, 1
# Justin Dealey 2, 1
# Stevenage railway station 2, 1
# regularly visiting a railway station 2, 1
# Nala 1
# donated to the RSPCA and Stevenage homelessness charity Feed Up Warm Up 1
# producer / writer 1

['Where will profit go?',
 'Who produced the song?',
 'What is the song called?',
 'Who gave the song its first play?',
 'When will the song be released?',
 'Who wrote the song?',
 'Where was the video filmed?',
 'How has nala been delighting commuters?',
 "Who's pictures went viral?",
 'All proceeds from the single will be what?',
 'What links Danny and Joe?']

In [26]:
answer_questions_with_confidence2(bbc_article_text, bbc_cat_questions) # 12 seconds

,confidence,question,answer
0,0.535661,Where will profit go?,RSPCA and Stevenage homelessness charity Feed Up Warm Up
1,0.981917,Who produced the song?,Danny Kirsch
2,0.883597,What is the song called?,Meow
3,0.992343,Who gave the song its first play?,Justin Dealey
4,0.852285,When will the song be released?,before Christmas
5,0.482969,Who wrote the song?,Danny Kirsch
6,0.920912,Where was the video filmed?,Stevenage railway station
7,0.444611,How has nala been delighting commuters?,taking photos of her at Stevenage station
8,0.617798,Who's pictures went viral?,A cat
9,0.024473,All proceeds from the single will be what?,Feed Up Warm Up


In [29]:
answer_questions_with_confidence(bbc_article_text, bbc_cat_questions) # 13 seconds

,confidence,question,answer
0,0.101562,Where will profit go?,RSPCA and Stevenage homelessness charity Feed Up Warm Up
1,0.963533,Who produced the song?,Danny Kirsch
2,0.819190,What is the song called?,Meow
3,0.781142,Who gave the song its first play?,Justin Dealey
4,0.682033,When will the song be released?,Wednesday
5,0.440879,Who wrote the song?,"Danny Kirsch, who wrote it with Joe Killington"
6,0.842697,Where was the video filmed?,Stevenage railway station
7,0.508643,How has nala been delighting commuters?,taking photos of her at Stevenage station
8,0.308242,Who's pictures went viral?,Nala
9,0.237600,All proceeds from the single will be what?,donated to the RSPCA and Stevenage homelessness charity Feed Up Warm Up


#### Chimp questions

In [32]:
chimp_questions = ['What is a typical group size in the wild?', 'Who is the son of Ai?', 'What can Ai do with the keyboard?',\
                   'What was Ai the first chimpanzee to do?']
chimp_answers = ['20', # 2
                 'Ayumu', # 2
                 'interact with a computer', 
                 'use arabic numerals to represent numbers'] # 2

In [33]:
answer_questions_with_confidence2(wikipedia_article_text, chimp_questions) # 13 seconds

,confidence,question,answer
0,0.491928,What is a typical group size in the wild?,20 chimpanzees
1,0.982641,Who is the son of Ai?,Ayumu
2,0.654109,What can Ai do with the keyboard?,paint and draw
3,0.643580,What was Ai the first chimpanzee to do?,learned to use Arabic numerals to represent numbers


In [35]:
answer_questions_with_confidence2(wikipedia_article_text, chimp_questions) # 15 seconds

,confidence,question,answer
0,0.491928,What is a typical group size in the wild?,20 chimpanzees
1,0.982641,Who is the son of Ai?,Ayumu
2,0.654109,What can Ai do with the keyboard?,paint and draw
3,0.643580,What was Ai the first chimpanzee to do?,learned to use Arabic numerals to represent numbers


## Example images

In [4]:
"""Example images for processing"""
"""Text"""
# Invoice
invoice = 'https://huggingface.co/spaces/impira/docquery/resolve/2359223c1837a7587402bda0f2643382a6eefeab/invoice.png'
# Simple poster
simple = 'https://www.11thhourracingteam.org/wp-content/uploads/11th-hour-racing-team-how-to-create-a-sustainability-policy-horizontal-3-1-1536x1056.png'
# Complex poster
complicated = 'https://cdn.greenmatch.co.uk/cdn-cgi/image/format=auto/2/2023/07/MAY23_4_02-Plastic-Waste_Global-Waste_2-1-663x1024.png'
# Microscopes text book page via web link
microscope = 'https://m.media-amazon.com/images/I/71Ts-QXYIhL._SL1500_.jpg'
# Magnification text book page downloaded to absolute file path
magnification = '/home/george/Downloads/magnification.jpg'

"""Handwriting"""
# Nice clear handwriting and cursive handwriting
clear = 'https://steemitimages.com/DQmcdbSGrnA9zeqWrYHD8EkNjvF9uxQCAeB7qnucUShpNDe/IMG_7345.PNG'
# Tricky handwriting
tricky = 'https://www.researchgate.net/profile/Neeta-Nain/publication/299666231/figure/fig1/AS:491693964304386@1494240384780/Example-image-of-a-general-handwritten-text-paragraph-from-IAM-dataset-4.png'
y5 = 'https://thelinksprimary.org.uk/wp-content/uploads/2023/10/Handwriting-Y6.png'

## Other ideas to explore

In [ ]:
"""These are possible ways to better process images"""
"""visual bert needs more configuring"""
https://huggingface.co/daki97/visualbert_finetuned_easy_vqa
https://huggingface.co/docs/transformers/model_doc/visual_bert#overview # overview is part of the url, not a comment
https://github.com/huggingface/transformers/blob/main/examples/research_projects/visual_bert/demo.ipynb
"""layout needs more configuring"""
https://huggingface.co/docs/transformers/model_doc/layoutlmv3
"""should work for extracting printed text, but only works for single lines"""
https://huggingface.co/microsoft/trocr-base-printed
"""suggestions on how to split into multiple lines for extracted text and handwriting"""
https://github.com/microsoft/unilm/issues/628
https://discuss.huggingface.co/t/trocr-fine-tuning/13293/3
"""vision encoder requires more configuration"""
https://huggingface.co/docs/transformers/model_doc/vision-encoder-decoder
"""Generate LaTEX from images"""
https://huggingface.co/Norm/nougat-latex-base
"""Automate snipping"""
https://support.techsmith.com/hc/en-us/articles/115002022732?ipc_item_name=snagit&ipc_platform=windows&utm_campaign=sw23&utm_medium=snagit&utm_source=product
https://www.movavi.com/learning-portal/snipping-tool-for-linux.html
https://screencloud.net/
https://www.codeproject.com/Articles/485883/Create-your-own-Snipping-Tool
https://www.codeinwp.com/blog/best-website-screenshot-tools/#gref
# Note it may not be possible to snip elements outside the browser from within the browser, for security reasons

## Exploration of retraining model

In [133]:
from transformers import TFAutoModelForSequenceClassification

In [134]:
question_answerer_train = TFAutoModelForSequenceClassification.from_pretrained('deepset/roberta-base-squad2', from_pt=True)

pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

2023-12-01 16:37:16.271633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-01 16:37:16.292324: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-01 16:37:16.292539: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (george-ThinkPad-X220-Tablet): /proc/driver/nvidia/version does not exist
2023-12-01 16:37:17.115447: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154414080 exceeds 10% of free system memory.
2023-12-01 16:37:17.564863: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154414080 exceeds 10% of free system memory.
2023-12-01 16:37:17.660538: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154414080 exceeds 10% of free system memory.
2023-12-01 16:

In [ ]:
question_answerer_train.